In [ ]:
import pandas as pd
from surprise import Dataset

# Cargar los datos
data = Dataset.load_builtin('ml-100k')

# Convertir a DataFrame
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
df


In [ ]:
from surprise import Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

# Cargar el dataset MovieLens
def load_movielens_data():
    data = Dataset.load_builtin('ml-100k')
    return data

# Cargar los títulos de las películas
def load_movie_titles():
    # Ajusta la ruta para incluir la carpeta "uk100"
    movie_titles = pd.read_csv('uk100/u.item', sep='|', encoding='latin-1', usecols=[0, 1], header=None)
    movie_titles.columns = ['movie_id', 'title']
    return movie_titles

# Entrenar el modelo de recomendación
def train_model(data):
    trainset, testset = train_test_split(data, test_size=0.2)
    algo = SVD()
    algo.fit(trainset)
    predictions = algo.test(testset)
    print("Rendimiento del modelo:")
    accuracy.rmse(predictions)
    accuracy.mae(predictions)  # Cálculo de MAE
    return algo, predictions, testset

# Realizar recomendaciones personalizadas
def recommend(algo, user_id, item_ids, n_recommendations=5):
    recommendations = []
    for item_id in item_ids:
        pred = algo.predict(user_id, item_id)
        recommendations.append((item_id, pred.est))
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations[:n_recommendations]

# Mostrar las recomendaciones con títulos
def display_recommendations(recommendations, movie_titles):
    print("\nRecomendaciones:")
    for movie_id, rating in recommendations:
        title = movie_titles[movie_titles['movie_id'] == int(movie_id)]['title'].values[0]
        print(f"Película: {title}, Rating estimado: {rating:.2f}")

if __name__ == "__main__":
    # Cargar los datos
    data = load_movielens_data()
    
    # Cargar los títulos de las películas
    movie_titles = load_movie_titles()
    
    # Entrenar el modelo
    algo, predictions, testset = train_model(data)
    
    # Listar los IDs de películas para un usuario (ejemplo)
    user_id = '196'
    all_item_ids = set(item for (_, item, _) in testset)  # Obtener películas en el set de prueba
    
    # Generar recomendaciones para el usuario 196
    top_recommendations = recommend(algo, user_id, all_item_ids, n_recommendations=5)
    display_recommendations(top_recommendations, movie_titles)


In [ ]:
from surprise import SVD, Dataset
from surprise.model_selection import GridSearchCV

# Cargar el dataset MovieLens
def load_movielens_data():
    data = Dataset.load_builtin('ml-100k')
    return data

# Buscar los mejores hiperparámetros para el modelo SVD
def find_best_hyperparameters(data):
    # Definir el espacio de búsqueda para los hiperparámetros
    param_grid = {
        'n_factors': [50, 100, 150, 200],  # Número de factores latentes
        'n_epochs': [20, 30, 50],          # Número de iteraciones en el entrenamiento
        'lr_all': [0.002, 0.005, 0.01],    # Tasa de aprendizaje
        'reg_all': [0.02, 0.1, 0.2, 0.4],  # Regularización
        'biased': [True, False]            # Si se deben usar sesgos (biases)
    }

    # Usar GridSearchCV para encontrar los mejores hiperparámetros
    gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=2)
    gs.fit(data)
    
    # Resultados del mejor modelo
    print("Mejores Hiperparámetros:")
    print(gs.best_params['rmse'])
    print(f"Mejor RMSE: {gs.best_score['rmse']}")
    print(f"Mejor MAE: {gs.best_score['mae']}")
    
    return gs.best_params['rmse'], gs.best_score['rmse'], gs.best_score['mae']

if __name__ == "__main__":
    # Cargar los datos
    data = load_movielens_data()

    # Encontrar los mejores hiperparámetros
    best_params, best_rmse, best_mae = find_best_hyperparameters(data)


In [ ]:
from surprise import Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

# Cargar el dataset MovieLens
def load_movielens_data():
    data = Dataset.load_builtin('ml-100k')
    return data

# Cargar los títulos de las películas
def load_movie_titles():
    # Ajusta la ruta para incluir la carpeta "uk100"
    movie_titles = pd.read_csv('uk100/u.item', sep='|', encoding='latin-1', usecols=[0, 1], header=None)
    movie_titles.columns = ['movie_id', 'title']
    return movie_titles

# Entrenar el modelo de recomendación con los mejores hiperparámetros
def train_model_with_best_params(data):
    # Dividir el dataset en entrenamiento y prueba
    trainset, testset = train_test_split(data, test_size=0.2)
    
    # Entrenar el modelo con los mejores hiperparámetros
    algo = SVD(n_factors=200, n_epochs=50, lr_all=0.01, reg_all=0.1, biased=True)
    algo.fit(trainset)
    
    # Evaluar el modelo en el conjunto de prueba
    predictions = algo.test(testset)
    
    return algo, predictions, testset

# Realizar recomendaciones personalizadas
def recommend(algo, user_id, item_ids, n_recommendations=5):
    recommendations = []
    for item_id in item_ids:
        pred = algo.predict(user_id, item_id)
        recommendations.append((item_id, pred.est))
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations[:n_recommendations]

# Mostrar las recomendaciones con títulos
def display_recommendations(recommendations, movie_titles):
    print("\nRecomendaciones:")
    for movie_id, rating in recommendations:
        title = movie_titles[movie_titles['movie_id'] == int(movie_id)]['title'].values[0]
        print(f"Película: {title}, Rating estimado: {rating:.2f}")

if __name__ == "__main__":
    # Cargar los datos
    data = load_movielens_data()
    
    # Cargar los títulos de las películas
    movie_titles = load_movie_titles()
    
    # Entrenar el modelo con los mejores hiperparámetros
    algo, predictions, testset = train_model_with_best_params(data)
    
    # Mostrar métricas de evaluación
    print("Evaluación del modelo con los mejores hiperparámetros:")
    print("RMSE:", accuracy.rmse(predictions, verbose=False))  # Solo imprimir RMSE
    print("MAE:", accuracy.mae(predictions, verbose=False))    # Solo imprimir MAE
    
    # Listar los IDs de películas para un usuario (ejemplo)
    user_id = '196'
    all_item_ids = set(item for (_, item, _) in testset)  # Obtener películas en el set de prueba
    
    # Generar recomendaciones para el usuario 196
    top_recommendations = recommend(algo, user_id, all_item_ids, n_recommendations=5)
    display_recommendations(top_recommendations, movie_titles)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Funciones para el análisis de datos y el modelo
def load_movielens_data():
    data = Dataset.load_builtin('ml-100k')
    return data

def train_model(data):
    trainset, testset = train_test_split(data, test_size=0.2)
    algo = SVD()
    algo.fit(trainset)
    predictions = algo.test(testset)
    print("Rendimiento del modelo:")
    accuracy.rmse(predictions)
    return algo, predictions, testset

# Funciones de visualización
def plot_rating_distribution(predictions):
    estimated_ratings = [pred.est for pred in predictions]
    plt.hist(estimated_ratings, bins=10, alpha=0.7, color='blue', edgecolor='black')
    plt.title("Distribución de Calificaciones Estimadas")
    plt.xlabel("Calificación Estimada")
    plt.ylabel("Frecuencia")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_actual_vs_predicted(predictions):
    actual_ratings = [pred.r_ui for pred in predictions]  # Calificaciones reales
    estimated_ratings = [pred.est for pred in predictions]  # Calificaciones estimadas
    
    plt.scatter(actual_ratings, estimated_ratings, alpha=0.6)
    plt.plot([min(actual_ratings), max(actual_ratings)], [min(actual_ratings), max(actual_ratings)], 'r--')
    plt.title("Comparación entre Calificaciones Reales y Estimadas")
    plt.xlabel("Calificación Real")
    plt.ylabel("Calificación Estimada")
    plt.grid(alpha=0.7)
    plt.show()

def plot_residuals(predictions):
    residuals = [pred.r_ui - pred.est for pred in predictions]  # Diferencia entre real y estimada
    
    plt.hist(residuals, bins=15, alpha=0.7, color='purple', edgecolor='black')
    plt.title("Distribución de los Errores de Predicción (Residuales)")
    plt.xlabel("Error de Predicción (Real - Estimado)")
    plt.ylabel("Frecuencia")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_ratings_per_user(data):
    df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
    ratings_per_user = df.groupby('user_id').size()
    
    plt.hist(ratings_per_user, bins=20, alpha=0.7, color='green', edgecolor='black')
    plt.title("Distribución del Número de Calificaciones por Usuario")
    plt.xlabel("Número de Calificaciones")
    plt.ylabel("Frecuencia")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_ratings_per_movie(data):
    df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
    ratings_per_movie = df.groupby('item_id').size()
    
    plt.hist(ratings_per_movie, bins=20, alpha=0.7, color='orange', edgecolor='black')
    plt.title("Distribución del Número de Calificaciones por Película")
    plt.xlabel("Número de Calificaciones")
    plt.ylabel("Frecuencia")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_avg_rating_per_user(data):
    df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
    avg_ratings_per_user = df.groupby('user_id')['rating'].mean()
    
    plt.hist(avg_ratings_per_user, bins=20, alpha=0.7, color='cyan', edgecolor='black')
    plt.title("Distribución de las Calificaciones Promedio por Usuario")
    plt.xlabel("Calificación Promedio")
    plt.ylabel("Frecuencia")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_avg_rating_per_movie(data):
    df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
    avg_ratings_per_movie = df.groupby('item_id')['rating'].mean()
    
    plt.hist(avg_ratings_per_movie, bins=20, alpha=0.7, color='red', edgecolor='black')
    plt.title("Distribución de las Calificaciones Promedio por Película")
    plt.xlabel("Calificación Promedio")
    plt.ylabel("Frecuencia")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_error_heatmap(predictions, top_n_users=50, top_n_items=50):
    errors = pd.DataFrame({
        'user_id': [pred.uid for pred in predictions],
        'item_id': [pred.iid for pred in predictions],
        'error': [abs(pred.r_ui - pred.est) for pred in predictions]
    })
    
    # Filtrar los usuarios y los ítems con más datos
    top_users = errors['user_id'].value_counts().head(top_n_users).index
    top_items = errors['item_id'].value_counts().head(top_n_items).index
    
    filtered_errors = errors[errors['user_id'].isin(top_users) & errors['item_id'].isin(top_items)]
    pivot_table = filtered_errors.pivot_table(index='user_id', columns='item_id', values='error', fill_value=0)
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_table, cmap='coolwarm', cbar=True, annot=False, fmt=".1f", linewidths=0.5)
    plt.title(f"Heatmap de Errores de Predicción: Top {top_n_users} Usuarios y {top_n_items} Ítems")
    plt.xlabel("Ítem")
    plt.ylabel("Usuario")
    plt.show()

# Bloque principal
if __name__ == "__main__":
    # Cargar datos
    data = load_movielens_data()
    
    # Entrenar el modelo y obtener predicciones
    algo, predictions, testset = train_model(data)
    
    # Generar gráficos
    plot_rating_distribution(predictions)
    plot_actual_vs_predicted(predictions)
    plot_residuals(predictions)
    plot_ratings_per_user(data)
    plot_ratings_per_movie(data)
    plot_avg_rating_per_user(data)
    plot_avg_rating_per_movie(data)
    plot_error_heatmap(predictions)
